In [5]:
pip install pipwin

Note: you may need to restart the kernel to use updated packages.


In [6]:
!pipwin install pyaudio

Package `pyaudio` found in cache
https://download.lfd.uci.edu/pythonlibs/x6hvwk7i/PyAudio-0.2.11-cp38-cp38-win_amd64.whl
PyAudio-0.2.11-cp38-cp38-win_amd64.whl


Traceback (most recent call last):
  File "c:\anaconda\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\anaconda\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\anaconda\Scripts\pipwin.exe\__main__.py", line 7, in <module>
  File "c:\anaconda\lib\site-packages\pipwin\command.py", line 103, in main
    cache.install(package)
  File "c:\anaconda\lib\site-packages\pipwin\pipwin.py", line 300, in install
    wheel_file = self.download(requirement)
  File "c:\anaconda\lib\site-packages\pipwin\pipwin.py", line 294, in download
    return self._download(requirement, dest)
  File "c:\anaconda\lib\site-packages\pipwin\pipwin.py", line 290, in _download
    obj.start()
  File "c:\anaconda\lib\site-packages\pySmartDL\pySmartDL.py", line 267, in start
    urlObj = urllib.request.urlopen(req, timeout=self.timeout, context=self.context)
  File "c:\anaconda\lib\urllib\request.py", line 222, in urlopen
    return open

In [7]:
!pip install python_speech_features

In [5]:
from tkinter import *
import ctypes,os
from tkinter.tix import *
from PIL import ImageTk, Image
import tkinter.filedialog as filedialog 
import tkinter.messagebox as tkMessageBox
import wave
import time
import pickle
import pyaudio,sqlite3
import warnings
import numpy as np
from sklearn import preprocessing
from scipy.io.wavfile import read
import python_speech_features as mfcc
from sklearn.mixture import GaussianMixture 
import threading as T
import shutil
from tkinter import ttk

warnings.filterwarnings("ignore")
os.makedirs('testing_set', exist_ok=True)
os.makedirs('training_set', exist_ok=True)

def Database():
    #Function For Database Connectivity
    global conn, cursor #Making conn,cursor a globle variable
    conn = sqlite3.connect("admin.db")#connecting to database
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS `users` (user_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,Name TEXT,Username TEXT, Password TEXT)")

def calculate_delta(array):
    rows,cols = array.shape
    print(rows)
    print(cols)
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i-j < 0:
              first =0
            else:
              first = i-j
            if i+j > rows-1:
                second = rows-1
            else:
                second = i+j 
            index.append((second,first))
            j+=1
        deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
    return deltas


def extract_features(audio,rate):
       
    mfcc_feature = mfcc.mfcc(audio,rate, 0.025, 0.01,20,nfft = 1200, appendEnergy = True)    
    mfcc_feature = preprocessing.scale(mfcc_feature)
    print(mfcc_feature)
    delta = calculate_delta(mfcc_feature)
    combined = np.hstack((mfcc_feature,delta)) 
    return combined


def train_model():

    source   = './training_set/'   
    dest = "./"
    train_file = "./training_set_addition.txt"        
    file_paths = open(train_file,'r')
    count = 1
    features = np.asarray(())
    for path in file_paths:    
        path = path.strip()   
        
        sr,audio = read(source + path)
        vector   = extract_features(audio,sr)
        
        if features.size == 0:
            features = vector  #count 
        else:
            features = np.vstack((features, vector)) # appending 

        if count == 1:    
            gmm = GaussianMixture(n_components = 6, max_iter = 200, covariance_type='diag',n_init = 3)
            gmm.fit(features)
            
            # dumping the trained gaussian model
            picklefile = path.split("-")[0]+".gmm"
            pickle.dump(gmm,open(dest + picklefile,'wb'))
            features = np.asarray(())
            count = 0
        count = count + 1


        
def voiceverificationlogin():
        #-----------------------------------------------------------------------
        loginwindow = Tk()
        loginwindow.title('LockTune')
        loginwindow.resizable(0,0)
        NAME = StringVar()
        #-----------------------------------------------------------------------
        user32 = ctypes.windll.user32 #processing for c 
        user32.SetProcessDPIAware()
        [w, h] = [user32.GetSystemMetrics(0), user32.GetSystemMetrics(1)] # specification 
        lt = [w, h]
        a = str(lt[0]//2-451)
        b= str(lt[1]//2-339)
        loginwindow.geometry("902x678+"+a+"+"+b) # pic , length ,width
        #-----------------------------------------------------------------------
        img = Image.open(r"C:\Users\sbpra\OneDrive\Desktop\locktone voice authentication\Images\Images\registerPage.png")
        img = ImageTk.PhotoImage(img)
        panel = Label(loginwindow, image=img)
        panel.pack(side="top", fill="both", expand="yes")# specification 
        #-----------------------------------------------------------------------
        def verify():
            n = (NAME.get()).lower() 
            if n=='':
               tkMessageBox.showinfo('LockTune','All Fields Are Mendatory To Fill')
            else:
                FORMAT = pyaudio.paInt16
                CHANNELS = 1
                RATE = 44100
                CHUNK = 512 # bites
                RECORD_SECONDS = 10
                device_index = 2
                audio = pyaudio.PyAudio() #initilize

                index = 1    

                stream = audio.open(format=FORMAT, channels=CHANNELS,
                                rate=RATE, input=True,input_device_index = index,
                                frames_per_buffer=CHUNK)

                Recordframes = [] # reading of each chunk
                for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)): 
                    data = stream.read(CHUNK)
                    Recordframes.append(data)
         
                stream.stop_stream()
                stream.close()
                audio.terminate()
                OUTPUT_FILENAME="sample.wav" # testing file  
                WAVE_OUTPUT_FILENAME=os.path.join("testing_set",OUTPUT_FILENAME) # comparing 
                trainedfilelist = open("testing_set_addition.txt", 'w')
                trainedfilelist.write(OUTPUT_FILENAME+"\n") 
                trainedfilelist.close()
                waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
                waveFile.setnchannels(CHANNELS)
                waveFile.setsampwidth(audio.get_sample_size(FORMAT))
                waveFile.setframerate(RATE)
                waveFile.writeframes(b''.join(Recordframes))
                waveFile.close()
                
                source   = "./testing_set/"  
                modelpath = "./"
                test_file = "./testing_set_addition.txt"       
                file_paths = open(test_file,'r')
                 
                gmm_files = [os.path.join(modelpath,fname) for fname in
                              os.listdir(modelpath) if fname.endswith('.gmm')] # directory
                 
                #Load the Gaussian gender Models
                models    = [pickle.load(open(fname,'rb')) for fname in gmm_files]
                speakers   = [fname.split("\\")[-1].split(".gmm")[0] for fname 
                              in gmm_files]
                 
                # Read the test directory and get the list of test audio files 

                for path in file_paths:  
                    path = path.strip()   

                    sr,audio = read(source + path)
                    vector   = extract_features(audio,sr)
                     
                    log_likelihood = np.zeros(len(models)) #binary form - bytes 
                    
                    for i in range(len(models)): 
                        gmm    = models[i]  #checking with each model one by one
                        scores = np.array(gmm.score(vector))
                        log_likelihood[i] = scores.sum()
                     
                    winner = np.argmax(log_likelihood)
                    if n in speakers[winner]:
                    
          
                        loginwindow.destroy()
                        home()
                        
                        
                    else:
     
                        tkMessageBox.showinfo('LockTune','Unable To Verify The User')
      

        def verifyv():
                x = T.Thread(target=verifyvoice)
                x.start()                                        
        photo = (Image.open(r"C:\Users\sbpra\OneDrive\Desktop\locktone voice authentication\Images\Images\voiceverificationlogin.png")).resize((300,300))
        img2 = ImageTk.PhotoImage(photo)
        b1 = Button(loginwindow, highlightthickness = 0, bd = 0,bg='white', image = img2,command=verify)
        b1.place(x=80,y=240)
        l1 = Label(loginwindow, text="Name         : ", bg="white",font = ('Arial',13,'bold'), fg="#2a72af")
        l1.place(x=100, y=240)    
        e1 = Entry(loginwindow, textvariable=NAME, bg="#e6f2ff",width = 17,font = ('Arial',13,'bold'), fg="#2a72af")
        e1.place(x=200, y=240)
        b2 = Button(loginwindow, text="LOGIN USING VOICE MANUAL METHOD", width="38", fg="#1E1D6C", highlightthickness = 0,relief = FLAT, bd = 0, borderwidth=0,font = ('Arial',9,'bold'), bg="white", command=signinmanual)
        b2.place(x=94, y=545)
        b3 = Button(loginwindow, text="NEW TO LOCKTUNE? REGISTER TO LOCKTUNE", width="38", fg="#1E1D6C", highlightthickness = 0,relief = FLAT, bd = 0, borderwidth=0,font = ('Arial',9,'bold'), bg="white", command=register)
        b3.place(x=94, y=570)
        loginwindow.mainloop()# stop in backgroud 
        
def signinmanual():
        #-----------------------------------------------------------------------
        mloginwindow = Toplevel()
        mloginwindow.title('LockTune')
        mloginwindow.resizable(0,0)
        #-----------------------------------------------------------------------
        user32 = ctypes.windll.user32
        user32.SetProcessDPIAware()
        [w, h] = [user32.GetSystemMetrics(0), user32.GetSystemMetrics(1)]
        lt = [w, h]
        a = str(lt[0]//2-451)
        b= str(lt[1]//2-339)
        mloginwindow.geometry("902x678+"+a+"+"+b)
        #-----------------------------------------------------------------------
        img = Image.open(r"C:\Users\sbpra\OneDrive\Desktop\locktone voice authentication\Images\Images\registerPage.png")
        img = ImageTk.PhotoImage(img)
        panel = Label(mloginwindow, image=img)
        panel.pack(side="top", fill="both", expand="yes")
        #-----------------------------------------------------------------------
        EMAIL = StringVar()
        PASSWORD = StringVar()
        def login():
            e,p=EMAIL.get(),PASSWORD.get()
            if e=='' or p=='':
                tkMessageBox.showinfo('LockTune','All Fields Are Mendatory To Fill')
            elif '@' not in e or '.' not in e:
                tkMessageBox.showinfo('LockTune','Please Enter A Valid Email Address')
                EMAIL.set('')
            elif len(p)<6:
                tkMessageBox.showinfo('LockTune','Password Length Must Be Greater Then Six')
                PASSWORD.set('')
            else:
                Database()
                cursor.execute(''' 
                                         SELECT * FROM users 
                                         WHERE(Username = ? and Password = ?)
                                         ''', (e,p))
                verifyLogin =cursor.fetchone()
                try:
                    if(e in verifyLogin and p in verifyLogin):
                        mloginwindow.destroy()
                        home()
                        tkMessageBox.showinfo(title="Login info", message="Logged In")
                        
                except:
                     tkMessageBox.showinfo(title="Login info", message="Unable To Login !!!")                 
        #-----------------------------------------------------------------------
        l1 = Label(mloginwindow, text="Email ID    : ", bg="white",font = ('Arial',13,'bold'), fg="#2a72af")
        l1.place(x=100, y=270)
        l2 = Label(mloginwindow, text="Password : ", bg="white",font = ('Arial',13,'bold'), fg="#2a72af")
        l2.place(x=100, y=320)
        e1 = Entry(mloginwindow, textvariable=EMAIL, bg="#e6f2ff",width = 17,font = ('Arial',13,'bold'), fg="#2a72af")
        e1.place(x=200, y=270)
        e2 = Entry(mloginwindow, textvariable=PASSWORD, bg="#e6f2ff",width = 17,font = ('Arial',13,'bold'), fg="#2a72af",show='*')
        e2.place(x=200, y=320)
        b1 = Button(mloginwindow, text="LOGIN", width="21", bg="#1E1D6C", highlightthickness = 0,relief = FLAT, bd = 1, borderwidth=0,font = ('Arial',15,'bold'), fg="#F5FAFE", command=login)
        b1.place(x=100, y=370)
        b2 = Button(mloginwindow, text="LOGIN USING VOICE VERIFICATION METHOD", width="38", fg="#1E1D6C", highlightthickness = 0,relief = FLAT, bd = 0, borderwidth=0,font = ('Arial',9,'bold'), bg="white", command=voiceverificationlogin)
        b2.place(x=94, y=430)
        b3 = Button(mloginwindow, text="NEW TO LOCKTUNE? REGISTER TO LOCKTUNE", width="38", fg="#1E1D6C", highlightthickness = 0,relief = FLAT, bd = 0, borderwidth=0,font = ('Arial',9,'bold'), bg="white", command=register)
        b3.place(x=94, y=470)
        mloginwindow.mainloop()

def register():
        #-----------------------------------------------------------------------
        regwindow = Toplevel()
        regwindow.title('LockTune')
        regwindow.resizable(0,0)
        #-----------------------------------------------------------------------
        user32 = ctypes.windll.user32
        user32.SetProcessDPIAware()
        [w, h] = [user32.GetSystemMetrics(0), user32.GetSystemMetrics(1)]
        lt = [w, h]
        a = str(lt[0]//2-451)
        b= str(lt[1]//2-339)
        regwindow.geometry("902x678+"+a+"+"+b)
        #-----------------------------------------------------------------------
        img = Image.open(r"C:\Users\sbpra\OneDrive\Desktop\locktone voice authentication\Images\Images\registerPage.png")
        img = ImageTk.PhotoImage(img)
        panel = Label(regwindow, image=img)
        panel.pack(side="top", fill="both", expand="yes")
        #-----------------------------------------------------------------------
        NAME = StringVar()
        EMAIL = StringVar()
        PASSWORD = StringVar()
        #-----------------------------------------------------------------------
        def addv():
            n,e,p=(NAME.get()).lower(),EMAIL.get(),PASSWORD.get()
            fname =  n+'.gmm'
            flist = os.listdir('./')
            if n=='' or e=='' or p=='':
                tkMessageBox.showinfo('LockTune','All Fields Are Mendatory To Fill')            
            else:
                Name =n

                FORMAT = pyaudio.paInt16
                CHANNELS = 1
                RATE = 44100
                CHUNK = 512
                RECORD_SECONDS = 10
                device_index = 2
                audio = pyaudio.PyAudio()
        ##      print("----------------------record device list---------------------")
        ##      info = audio.get_host_api_info_by_index(0)
        ##      numdevices = info.get('deviceCount')
        ##      for i in range(0, numdevices):
        ##              if (audio.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
        ##                  print("Input Device id ", i, " - ", audio.get_device_info_by_host_api_device_index(0, i).get('name'))
        ##      print("-------------------------------------------------------------")
                index = 1   
                stream = audio.open(format=FORMAT, channels=CHANNELS,
                                rate=RATE, input=True,input_device_index = index,
                                frames_per_buffer=CHUNK)
                b1["text"]="Speak Now"
                b1.config(command=None)
                Recordframes = []
                for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
                    data = stream.read(CHUNK)
                    Recordframes.append(data)
                b1["text"]="Audio Sample Taken"
                
                
                OUTPUT_FILENAME=Name+"-sample"+".wav"
                WAVE_OUTPUT_FILENAME=os.path.join("training_set",OUTPUT_FILENAME)

                trainedfilelist = open("training_set_addition.txt", 'w')
                trainedfilelist.write(OUTPUT_FILENAME+"\n")
                trainedfilelist.close()
                stream.stop_stream()
                stream.close()
                audio.terminate()
                waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
                waveFile.setnchannels(CHANNELS)
                waveFile.setsampwidth(audio.get_sample_size(FORMAT))
                waveFile.setframerate(RATE)
                waveFile.writeframes(b''.join(Recordframes))
                waveFile.close()
                train_model()
        #-----------------------------------------------------------------------
        def addvoice():# adding of voice 
                x = T.Thread(target=addv)
                x.start()            
        def reg():
            n,e,p=(NAME.get()).lower(),EMAIL.get(),PASSWORD.get()
            fname =  n+'.gmm'
            flist = os.listdir('./')
            if n=='' or e=='' or p=='':
                tkMessageBox.showinfo('LockTune','All Fields Are Mendatory To Fill')
            elif '@' not in e or '.' not in e:
                tkMessageBox.showinfo('LockTune','Please Enter A Valid Email Address')
                EMAIL.set('')
            elif len(p)<6:
                tkMessageBox.showinfo('LockTune','Password Length Must Be Greater Then Six')
                PASSWORD.set('')
            elif fname not in flist:
                 tkMessageBox.showinfo('LockTune','Please Add Voice Sample')
            else:
                Database()
                cursor.execute('''INSERT INTO users(Name, Username, Password) VALUES(?,?,?)''' , (n,e, p))
                conn.commit() #access 
                tkMessageBox.showinfo('LockTune','User Registered Successfully') 
                regwindow.destroy()
                voiceverificationlogin()
                
        #-----------------------------------------------------------------------
        l1 = Label(regwindow, text="Name         : ", bg="white",font = ('Arial',13,'bold'), fg="#2a72af")
        l1.place(x=100, y=270)    
        l2 = Label(regwindow, text="Email ID    : ", bg="white",font = ('Arial',13,'bold'), fg="#2a72af")
        l2.place(x=100, y=320)
        l3 = Label(regwindow, text="Password : ", bg="white",font = ('Arial',13,'bold'), fg="#2a72af")
        l3.place(x=100, y=370)
        e1 = Entry(regwindow, textvariable=NAME, bg="#e6f2ff",width = 17,font = ('Arial',13,'bold'), fg="#2a72af")
        e1.place(x=200, y=270)
        e2 = Entry(regwindow, textvariable=EMAIL, bg="#e6f2ff",width = 17,font = ('Arial',13,'bold'), fg="#2a72af")
        e2.place(x=200, y=320)
        e3 = Entry(regwindow, textvariable=PASSWORD, bg="#e6f2ff",width = 17,font = ('Arial',13,'bold'), fg="#2a72af")
        e3.place(x=200, y=370) 
        b1 = Button(regwindow, text="ADD THE VOICE SAMPLE", width="21", bg="#1E1D6C", highlightthickness = 0,relief = FLAT, bd = 1, borderwidth=0,font = ('Arial',15,'bold'), fg="#F5FAFE", command=addvoice)
        b1.place(x=100, y=420)
        b2 =  Button(regwindow, text="REGISTER", width="21", bg="#1E1D6C", highlightthickness = 0,relief = FLAT, bd = 1, borderwidth=0,font = ('Arial',15,'bold'), fg="#F5FAFE", command=reg)
        b2.place(x=100, y=480)
        b3 = Button(regwindow, text="ALREADY REGISTERD? LOGIN TO LOCKTUNE", width="38", fg="#1E1D6C", highlightthickness = 0,relief = FLAT, bd = 0, borderwidth=0,font = ('Arial',9,'bold'), bg="white", command=voiceverificationlogin)
        b3.place(x=94, y=550)
        regwindow.mainloop()

def home():
        #-----------------------------------------------------------------------
        home = Toplevel()#Tk()
        home.title('LockTune')
        home.resizable(0,0)
        #-----------------------------------------------------------------------
        user32 = ctypes.windll.user32
        user32.SetProcessDPIAware()
        [w, h] = [user32.GetSystemMetrics(0), user32.GetSystemMetrics(1)]
        lt = [w, h]
        a = str(lt[0]//2-451) #no. of char's
        b= str(lt[1]//2-339)
        home.geometry("902x678+"+a+"+"+b)#interface size 
        #-----------------------------------------------------------------------
        img = Image.open(r"C:\Users\sbpra\OneDrive\Desktop\locktone voice authentication\Images\Images\homepage.png")
        img = ImageTk.PhotoImage(img)
        panel = Label(home, image=img)
        panel.pack(side="top", fill="both", expand="yes")
        def addnew():
                filename = filedialog.askopenfilename(initialdir=os.getcwd(), title="Select file", filetypes=(("All Files", "*.*"),("Text Files",(".txt"))))
                os.makedirs('locktone_safe', exist_ok=True)
                shutil.move(filename,'./locktone_safe')
                ctypes.windll.kernel32.SetFileAttributesW('locktone_safe', 2)
                tkMessageBox.showinfo('LockTune','File Added Successfully !!!')
        def view():
                viewform = Toplevel()
                viewform.title("LockTone File Safe")
                viewform.geometry('580x600')
                viewform.config(bg='#052767')
                viewform.resizable(0, 0)
                
                def show():
                    df=os.listdir('locktone_safe')
                    index = len(df) 
                    for i in range(0,index): #checking 
                          nd = df[i]
                          
                          tree.insert('', 'end', values=(nd))
                          
                #Creating A View Item Window

                TopViewForm = Frame(viewform, width=600, bd=1, relief=SOLID)
                TopViewForm.pack(side=TOP, fill=X)
                

                MidViewForm = Frame(viewform, width=600,bg='#052767')
                MidViewForm.pack()


                lbl_text = Label(TopViewForm, text="LockTone File Safe", font=('Times New Roman', 24), width=600, bg="#052767",fg='white')
                lbl_text.pack(fill=X)
                
                
                scrollbarx = Scrollbar(MidViewForm, orient=HORIZONTAL)
                scrollbary = Scrollbar(MidViewForm, orient=VERTICAL)
                


                tree = ttk.Treeview(MidViewForm,
                                    columns=('filename'),
                                    selectmode="extended", height=23, yscrollcommand=scrollbary.set, xscrollcommand=scrollbarx.set)

                scrollbary.config(command=tree.yview)
                scrollbary.pack(side=RIGHT, fill=Y)
                scrollbarx.config(command=tree.xview)
                scrollbarx.pack(side=BOTTOM, fill=X)

                style = ttk.Style(MidViewForm)
                # set ttk theme to "clam" which support the fieldbackground option
                style.theme_use("clam")    
                style.configure("Treeview", background="white",fieldbackground="white", foreground="#052767")
                tree.heading('filename', text="FileNames",anchor=W)
                tree.column('#0', stretch=NO, minwidth=0, width=0)
                tree.column('#1', stretch=NO, minwidth=559, width=559)
                tree.pack()
                def openfile():
                    if not tree.selection():
                        tkMessageBox.showinfo('LockTune','Please Select Any File First')

                    else:

                            curItem = tree.focus()
                            contents =(tree.item(curItem))

                            selecteditem = contents['values']
                            

                            direc = './locktone_safe/'+selecteditem[0]
                            os.system("start " + direc)
                Button(viewform,text = 'OPEN FILE', width="40", bg="#F5FAFE", highlightthickness = 0,relief = FLAT, bd = 1, borderwidth=2,font = ('Arial',18,'bold'), fg="#052767", command=openfile).pack(side=BOTTOM)
                show()
                viewform.mainloop()
                
        def remove():
                viewform = Toplevel()
                viewform.title("LockTone File Safe")
                viewform.geometry('580x600')
                viewform.config(bg='#052767')
                viewform.resizable(0, 0)
                
                def show():
                    df=os.listdir('locktone_safe')
                    index = len(df)
                    for i in range(0,index):
                          nd = df[i]
                          
                          tree.insert('', 'end', values=(nd))
                          
                #Creating A View Item Window

                TopViewForm = Frame(viewform, width=600, bd=1, relief=SOLID)
                TopViewForm.pack(side=TOP, fill=X)
                

                MidViewForm = Frame(viewform, width=600,bg='#052767')
                MidViewForm.pack()


                lbl_text = Label(TopViewForm, text="LockTone File Safe", font=('Times New Roman', 24), width=600, bg="#052767",fg='white')
                lbl_text.pack(fill=X)
                
                
                scrollbarx = Scrollbar(MidViewForm, orient=HORIZONTAL)
                scrollbary = Scrollbar(MidViewForm, orient=VERTICAL)
                


                tree = ttk.Treeview(MidViewForm,
                                    columns=('filename'),
                                    selectmode="extended", height=23, yscrollcommand=scrollbary.set, xscrollcommand=scrollbarx.set)

                scrollbary.config(command=tree.yview)
                scrollbary.pack(side=RIGHT, fill=Y)
                scrollbarx.config(command=tree.xview)
                scrollbarx.pack(side=BOTTOM, fill=X)

                style = ttk.Style(MidViewForm)
                # set ttk theme to "clam" which support the fieldbackground option
                style.theme_use("clam")    
                style.configure("Treeview", background="white",fieldbackground="white", foreground="#052767")
                tree.heading('filename', text="FileNames",anchor=W)
                tree.column('#0', stretch=NO, minwidth=0, width=0)
                tree.column('#1', stretch=NO, minwidth=559, width=559)
                tree.pack()
                def removefile():
                    if not tree.selection():
                        tkMessageBox.showinfo('LockTune','Please Select Any File First')

                    else:
                        result = tkMessageBox.askquestion('LockTone', 'Are you sure you want to delete this file permanently ?', icon="warning")

                        if result == 'yes':
                            curItem = tree.focus()
                            contents =(tree.item(curItem))
                            selecteditem = contents['values']
                            direc = './locktone_safe/'+selecteditem[0]
                            os.remove(direc)
                            tree.delete(curItem)
                            
                Button(viewform,text = 'DELETE FILE', width="40", bg="#F5FAFE", highlightthickness = 0,relief = FLAT, bd = 1, borderwidth=2,font = ('Arial',18,'bold'), fg="#052767", command=removefile).pack(side=BOTTOM)
                show()
                viewform.mainloop()
                
        def about():
                #-----------------------------------------------------------------------
                about = Toplevel()
                about.title('LockTune')
                about.resizable(0,0)
                #-----------------------------------------------------------------------
                user32 = ctypes.windll.user32
                user32.SetProcessDPIAware()
                [w, h] = [user32.GetSystemMetrics(0), user32.GetSystemMetrics(1)]
                lt = [w, h]
                a = str(lt[0]//2-451)
                b= str(lt[1]//2-339)
                about.geometry("902x678+"+a+"+"+b)
                #-----------------------------------------------------------------------
                img = Image.open(r"C:\Users\sbpra\OneDrive\Desktop\locktone voice authentication\Images\Images\about.png")
                img = ImageTk.PhotoImage(img)
                panel = Label(about, image=img)
                panel.pack(side="top", fill="both", expand="yes")
                about.mainloop()
        def contact():
                #-----------------------------------------------------------------------
                contact = Toplevel()
                contact.title('LockTune')
                contact.resizable(0,0)
                #-----------------------------------------------------------------------
                user32 = ctypes.windll.user32
                user32.SetProcessDPIAware()
                [w, h] = [user32.GetSystemMetrics(0), user32.GetSystemMetrics(1)]
                lt = [w, h]
                a = str(lt[0]//2-451)
                b= str(lt[1]//2-339)
                contact.geometry("902x678+"+a+"+"+b)
                #-----------------------------------------------------------------------
                img = Image.open(r"images/contactus.png")
                img = ImageTk.PhotoImage(img)
                panel = Label(contact, image=img)
                panel.pack(side="top", fill="both", expand="yes")
                contact.mainloop()
        def logout():
            home.destroy()
            voiceverificationlogin()
        b1 = Button(home, text="ADD NEW FILE", width="21", bg="#1E1D6C", highlightthickness = 0,relief = FLAT, bd = 1, borderwidth=0,font = ('Arial',15,'bold'), fg="#F5FAFE", command=addnew)
        b1.place(x=80, y=320)
        b_ab =Button(home, text="ABOUT US", width="9", fg="#052767", highlightthickness = 0,relief = FLAT, bd = 1, borderwidth=1,font = ('Arial',13,'bold'), bg="#ffffff", command=about)
        b_ab.place(x=150, y=37)
        b_contact = Button(home, text="CONTACT US", width="11", fg="#052767", highlightthickness = 0,relief = FLAT, bd = 1, borderwidth=1,font = ('Arial',13,'bold'), bg="#ffffff", command=contact)
        b_contact.place(x=250, y=37)
        b2 =  Button(home, text="OPEN FILE", width="21", fg="#1E1D6C", highlightthickness = 0,relief = FLAT, bd = 1, borderwidth=0,font = ('Arial',15,'bold'), bg="#F5FAFE", command=view)
        b2.place(x=80, y=380)
        b3 = Button(home, text="REMOVE FILE", width="21", fg="#1E1D6C", highlightthickness = 0,relief = FLAT, bd = 1, borderwidth=0,font = ('Arial',15,'bold'), bg="#F5FAFE", command=remove)
        b3.place(x=80, y=440)
        b4 =  Button(home, text="LOGOUT", width="21", bg="#1E1D6C", highlightthickness = 0,relief = FLAT, bd = 1, borderwidth=0,font = ('Arial',15,'bold'), fg="#F5FAFE", command=logout)
        b4.place(x=80, y=500)
        
        home.mainloop()
        
voiceverificationlogin()

[[-3.71824024 -5.38489779  3.3471033  ... -1.28707533  1.62904237
   0.35722211]
 [-3.69979815 -4.93945901  3.85368461 ... -1.08678598  1.92440398
   0.93449296]
 [-3.69317321 -5.03668435  3.75548937 ... -0.86374519  1.64212114
   0.67268145]
 ...
 [-0.70830214 -0.45123889  0.22118482 ... -0.26622997  0.12495553
   0.27365797]
 [-0.75181722 -0.33983359  0.5044857  ...  0.645367    2.18857597
   2.55118864]
 [-0.95676176 -0.37395648  0.47691138 ...  0.13664388  1.87286282
   1.48912554]]
999
20


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\anaconda\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "<ipython-input-5-625ce8a9ef49>", line 556, in removefile
    os.remove(direc)
FileNotFoundError: [WinError 2] The system cannot find the file specified: './locktone_safe/Chill'
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\anaconda\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "<ipython-input-5-625ce8a9ef49>", line 556, in removefile
    os.remove(direc)
FileNotFoundError: [WinError 2] The system cannot find the file specified: './locktone_safe/Chill'
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\anaconda\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "<ipython-input-5-625ce8a9ef49>", line 603, in logout
    voiceverificationlogin()
  File "<ipython-input-5-625ce8a9ef49>", line 115, in 